<a href="https://colab.research.google.com/github/Nivedhasriram/Machine-Learning-and-Algorithms/blob/main/LAB_9(Feature_Selection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold, SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [49]:
#1. Missing value ratio

In [50]:
df_diabetes = pd.read_csv('diabetes.csv')
df_melbourne = pd.read_csv('melbourne_housing_raw.csv')

In [51]:
miss_per = df_diabetes.isnull().mean() * 100
df_reduced = df_diabetes.loc[:, miss_per < 30]

In [52]:
x = df_reduced.drop('Outcome', axis=1)
y = df_reduced['Outcome']

In [53]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [54]:
model = LogisticRegression()
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [55]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy after using missing value ratio: {accuracy}")

Accuracy after using missing value ratio: 0.7467532467532467


In [56]:
target = df_melbourne['Price']
miss_per = df_melbourne.isnull().mean() * 100
df_mel_reduced = df_melbourne.loc[:, (miss_per < 20) | (miss_per.index == 'Price')]
df_mel_reduced = df_mel_reduced.dropna()

In [57]:
x_melb = df_mel_reduced.drop('Price', axis=1)
y_melb = df_mel_reduced['Price']

In [58]:
x_melb_numeric = x_melb.select_dtypes(exclude=['object'])
x_train_melb, x_test_melb, y_train_melb, y_test_melb = train_test_split(
    x_melb_numeric, y_melb, test_size=0.2, random_state=42
)

In [59]:
model_melb = LinearRegression()
model_melb.fit(x_train_melb, y_train_melb)

LinearRegression()

In [60]:
y_pred_melb = model_melb.predict(x_test_melb)
rmse = mean_squared_error(y_test_melb, y_pred_melb, squared=False)
print(f"RMSE after using missing value ratio: {rmse}")

RMSE after using missing value ratio: 500220.9955817846


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [61]:
#2. High Correlation filter

In [62]:
corr_matrix_d = df_diabetes.corr().abs()

In [63]:
u_d = corr_matrix_d.where(np.triu(np.ones(corr_matrix_d.shape), k=1).astype(bool))
high_corr_d = [column for column in u_d.columns if any(u_d[column] > 0.8)]

In [64]:
df_diabetes_reduced = df_diabetes.drop(columns=high_corr_d)

In [65]:
x_d = df_diabetes_reduced.drop('Outcome', axis=1)
y_d = df_diabetes_reduced['Outcome']

In [66]:
x_train_d, x_test_d, y_train_d, y_test_d = train_test_split(x_d, y_d, test_size=0.2, random_state=42)

In [67]:
model_d = LogisticRegression()
model_d.fit(x_train_d, y_train_d)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [68]:
y_pred_d = model_d.predict(x_test_d)
accuracy_d = accuracy_score(y_test_d, y_pred_d)
print(f"Accuracy after using high correlation filter: {accuracy_d}")

Accuracy after using high correlation filter: 0.7467532467532467


In [69]:
#3.Low Variance Filter

In [70]:
selector = VarianceThreshold(threshold=0.01)
x_diabetes_reduced = selector.fit_transform(x_d)

In [71]:
x_train_d, x_test_d, y_train_d, y_test_d = train_test_split(x_diabetes_reduced, y_d, test_size=0.2, random_state=42)

In [72]:
model_d = LogisticRegression(max_iter=1000)
model_d.fit(x_train_d, y_train_d)

LogisticRegression(max_iter=1000)

In [73]:
y_pred_d=model_d.predict(x_test_d)
accuracy_d = accuracy_score(y_test_d, y_pred_d)
print(f"Accuracy after using low variance filter: {accuracy_d}")

Accuracy after using low variance filter: 0.7467532467532467


In [74]:
#4.Forward Feature Selection

In [75]:
scaler = StandardScaler()
x_diabetes_scaled = scaler.fit_transform(x_d)

In [76]:
x_train_d, x_test_d, y_train_d, y_test_d = train_test_split(x_diabetes_scaled, y_d, test_size=0.2, random_state=42)

In [77]:
model_d = LogisticRegression(max_iter=1000)

In [78]:
sfs_d = SequentialFeatureSelector(model_d, n_features_to_select="auto", direction="forward")
sfs_d.fit(x_train_d, y_train_d)

SequentialFeatureSelector(estimator=LogisticRegression(max_iter=1000))

In [79]:
selected_features_d = sfs_d.get_support(indices=True)
print(f"Selected features after forward feature selection: {selected_features_d}")

Selected features after forward feature selection: [0 1 2 5]


In [80]:
x_train_d_selected = x_train_d[:, selected_features_d]
x_test_d_selected = x_test_d[:, selected_features_d]

In [81]:
model_d.fit(x_train_d_selected, y_train_d)

LogisticRegression(max_iter=1000)

In [82]:
y_pred_d = model_d.predict(x_test_d_selected)
accuracy_d = accuracy_score(y_test_d, y_pred_d)
print(f"Accuracy after forward feature selection: {accuracy_d}")

Accuracy after forward feature selection: 0.7792207792207793


In [91]:
#5. Backward Feature Selection

In [84]:
model_d = DecisionTreeClassifier(random_state=42)

In [85]:
sfs_d = SequentialFeatureSelector(model_d, n_features_to_select="auto", direction="forward")
sfs_d.fit(x_train_d, y_train_d)

SequentialFeatureSelector(estimator=DecisionTreeClassifier(random_state=42))

In [92]:
selected_features_d = sfs_d.get_support(indices=True)
print(f"Selected features after backward feature selection: {selected_features_d}")

Selected features after backward feature selection: [1 2 3 5]


In [88]:
x_train_d_selected = x_train_d[:, selected_features_d]
x_test_d_selected = x_test_d[:, selected_features_d]

In [89]:
model_d.fit(x_train_d_selected, y_train_d)

DecisionTreeClassifier(random_state=42)

In [93]:
y_pred_d = model_d.predict(x_test_d_selected)
accuracy_d = accuracy_score(y_test_d, y_pred_d)
print(f"Accuracy after backward feature selection: {accuracy_d}")

Accuracy after backward feature selection: 0.6428571428571429


In [94]:
#6.Random Forest

In [98]:
model_diabetes = RandomForestClassifier(n_estimators=100, random_state=42)
model_diabetes.fit(x_train, y_train)
features = model_diabetes.feature_importances_

In [99]:
feature_name = df_diabetes.drop('Outcome', axis=1).columns
imp_df = pd.DataFrame({'Feature': feature_name, 'Importance': features})

In [101]:
imp_df = imp_df.sort_values(by='Importance', ascending=False)
print("Feature importance scores:\n",imp_df)

Feature importance scores:
                     Feature  Importance
1                   Glucose    0.258864
5                       BMI    0.169984
7                       Age    0.140931
6  DiabetesPedigreeFunction    0.123768
2             BloodPressure    0.088134
0               Pregnancies    0.076551
4                   Insulin    0.076122
3             SkinThickness    0.065646


In [102]:
top_5_features = imp_df['Feature'].head(5).values
print("Top 5 features:", top_5_features)

Top 5 features: ['Glucose' 'BMI' 'Age' 'DiabetesPedigreeFunction' 'BloodPressure']


In [103]:
x_train_d_top_5 = x_train_d[:, imp_df.index[:5]]
x_test_d_top_5 = x_test_d[:, imp_df.index[:5]]

In [104]:
model_d_top_5 = RandomForestClassifier(n_estimators=100, random_state=42)
model_d_top_5.fit(x_train_d_top_5, y_train_d)

RandomForestClassifier(random_state=42)

In [105]:
y_pred_d_top_5 = model_d_top_5.predict(x_test_d_top_5)
accuracy_d_top_5 = accuracy_score(y_test_d, y_pred_d_top_5)
print(f"Accuracy after using random forest: {accuracy_d_top_5}")

Accuracy after using random forest: 0.7857142857142857
